<a href="https://colab.research.google.com/github/kristynpantoja/math689project/blob/master/pytorchM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
!pip install torchvision

    100% |████████████████████████████████| 61kB 3.4MB/s 
    100% |████████████████████████████████| 2.0MB 9.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


Model Parameters

In [3]:
!pip install gensim

    100% |████████████████████████████████| 23.6MB 1.4MB/s 
    100% |████████████████████████████████| 1.4MB 12.0MB/s 
    100% |████████████████████████████████| 133kB 18.3MB/s 
    100% |████████████████████████████████| 61kB 16.1MB/s 
    100% |████████████████████████████████| 4.8MB 3.1MB/s 
    100% |████████████████████████████████| 552kB 15.5MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
# from torchvision import datasets, transforms
# from torchvision.utils import save_image

from types import SimpleNamespace

from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import Word2Vec, KeyedVectors

Data: 20newsgroups
We get the document-term matrix

In [0]:
# categories = ['talk.politics.guns', 'sci.space', 'soc.religion.christian',
#               'misc.forsale', 'rec.sport.baseball', 'comp.sys.mac.hardware']
categories = ['talk.politics.guns', 'sci.space']
# newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_train = fetch_20newsgroups(subset='train')
vectorizer = CountVectorizer(stop_words = 'english', min_df=.01, max_df=0.9, 
                             token_pattern = u'(?ui)\\b[a-z]{3,}\\b')
count_vecs = vectorizer.fit_transform(newsgroups_train.data)
doc_term_matrix = count_vecs.toarray()
doc_term_matrix.shape # number of documents, number of words (in vocab)
tokenizer = vectorizer.build_tokenizer()

doc_term_matrix_tensor = torch.from_numpy(doc_term_matrix).float()

In [22]:
doc_term_matrix.shape # vocab size

(1139, 2441)

In [0]:
len(vectorizer.get_feature_names())

In [0]:
dict_word_freq = dict(zip(vectorizer.get_feature_names(), list(doc_term_matrix.sum(0))))

ragged array of words in each document (by index in vocabulary)

In [0]:
# def countsToInput(row):
#   return np.repeat(np.arange(doc_term_matrix.shape[1]),row)
  
# def numWords(row):
#   return row.sum()

# N_train = np.apply_along_axis(numWords, axis=1, arr=doc_term_matrix)
# data_train = []
# for d in range(doc_term_matrix.shape[0]):
#   data_train.append(torch.from_numpy(countsToInput(doc_term_matrix[d])))

## Word2vec

In [7]:
# ideally, we would do some preprocessing
newsgroups_train_preproc = []
for document in newsgroups_train.data:
    newsgroups_train_preproc.append(document.split())

# make the model
w2v = Word2Vec(sg=1, negative=5, size=100, window=10, min_count=1, max_vocab_size=None, max_final_vocab=None)
# w2v.build_vocab(newsgroups_train_preproc)
w2v.build_vocab_from_freq(word_freq = dict_word_freq)
# train the model
w2v.train(sentences=newsgroups_train_preproc,epochs=10, total_examples=doc_term_matrix.shape[1])
# save the model
# w2v.save("sg_1_M2")

(625260, 3791210)

In [0]:
len(set([item for sublist in newsgroups_train_preproc for item in sublist]))

In [0]:
  torch.tensor(w2v.syn1neg).shape

In [0]:
len(w2v.wv.vocab)

In [0]:
doc_term_matrix.shape[1] == len(w2v.wv.vocab)

In [0]:
w2v.syn1neg

In [0]:
w2v.wv.most_similar("university")

## Setup for Model

In [0]:
# from types import SimpleNamespace
# args_dict = {"batch_size" : 50, "epochs" : 50, "no_cuda" : False, "seed" : 1, "log_interval" : 10}
# args = SimpleNamespace(**args_dict)
# args.epochs
# args.cuda = not args.no_cuda and torch.cuda.is_available()

args_dict = {"batch_size" : 50, "epochs" : 30, "no_cuda" : False, "seed" : 1, 
             "log_interval" : 100, "nogpu" : True}
args = SimpleNamespace(**args_dict)
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.ToTensor()),
#     batch_size=args.batch_size, shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
#     batch_size=args.batch_size, shuffle=True, **kwargs)





Define model

In [0]:
class VAE(nn.Module):
    def __init__(self, vocab_size, num_docs, wordvec_dim, encoder_hidden, rp_normal_dim, num_samples, K):
        super(VAE, self).__init__()
        
        self.vocab_size = vocab_size
        self.num_samples = num_samples
        self.num_docs = num_docs
        self.K = K
        
        self.word_embedding = nn.Embedding(vocab_size, wordvec_dim) # decoder
#         self.word_embedding = nn.Embedding.from_pretrained(torch.tensor(w2v.syn1neg), freeze=True)
        self.topic_embedding = nn.Embedding(K, wordvec_dim) # decoder
        self.lin1 = nn.Linear(vocab_size, encoder_hidden) # encoder
        self.mean = nn.Linear(encoder_hidden, rp_normal_dim) # encoder
#         self.mean_bn = nn.BatchNorm1d(rp_normal_dim)
        self.logvar = nn.Linear(encoder_hidden, rp_normal_dim) # encoder
#         self.logvar_bn = nn.BatchNorm1d(rp_normal_dim)
        self.lin2 = nn.Linear(rp_normal_dim, K) # decoder 
        self.dropout = nn.Dropout(p=0.8)
        
        self.beta = torch.zeros([vocab_size, K], dtype = torch.float32) # decoder
#         self.theta = torch.zeros([10, K], dtype = torch.float32)

        ###
#         self.decoder = nn.Linear(K, vocab_size) ### vocab_size == 1995 in the paper
#         self.decoder_bn = nn.BatchNorm1d(vocab_size) 
        ###

        # prior mean and variance as constant buffers
        prior_mean   = torch.Tensor(1, rp_normal_dim).fill_(0)
        prior_var    = torch.Tensor(1, rp_normal_dim).fill_(0.995)
        prior_logvar = prior_var.log()
        self.register_buffer('prior_mean',    prior_mean)
        self.register_buffer('prior_var',     prior_var)
        self.register_buffer('prior_logvar',  prior_logvar)

    def encode(self, x):
        h1 = F.relu(self.lin1(x))
        h2 = self.dropout(h1)
#         return self.mean_bn(self.mean(h2)), self.mean_bn(self.logvar(h2))
        return self.mean(h2), self.logvar(h2)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar) # get sigma
        eps = torch.randn_like(std) # get epsilon, generated from N(0, I_k) where k is dimension of std: k x 1
        return eps.mul(std).add_(mu) # this gives x ~ N(mu, var)
        # note: .mul is element-wise multiplication - this is fine, since sigma is diagonal matrix

      
    def decode(self, z, docs):
        x = self.lin2(z) 
        theta = F.softmax(x, dim = 1) # to get theta, dim = batch size x K
        
        ###
#         recon = F.softmax(self.decoder_bn(self.decoder(theta))) # batch size x V
#         return recon
        ###
        
        self.beta = F.softmax(self.word_embedding.weight.mm(self.topic_embedding.weight.t()), dim = 0) # beta, dim = V x K
#         print(theta)
#         print(self.beta)
#         print(docs.mm(self.beta).mm(theta.t()).diag())

#         log_theta_dot_beta = docs.mm((self.beta.mm(theta.t())).log()) # dim is B x B
        log_theta_dot_beta = theta.mm(self.beta.t()) # output is B x V
        return log_theta_dot_beta
#         return log_theta_dot_beta.sum()
        
        
    def forward(self, docs):
        mu, logvar = self.encode(docs)
        log_p = torch.zeros(1)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z, docs)
        return recon, self.loss(docs, recon, mu, logvar, logvar.exp())
      
      
    def loss(self, docs, recon, posterior_mean, posterior_logvar, posterior_var, avg=True):
      # NL
      NL  = -(docs * (recon+1e-10).log()).sum(1) # vector with batch-size number of elements
      # KLD, see Section 3.3 of Akash Srivastava and Charles Sutton, 2017, 
      # https://arxiv.org/pdf/1703.01488.pdf
      prior_mean   = Variable(self.prior_mean).expand_as(posterior_mean) # batch-size x num_topics
      prior_var    = Variable(self.prior_var).expand_as(posterior_mean)
      prior_logvar = Variable(self.prior_logvar).expand_as(posterior_mean)
      var_division    = posterior_var  / prior_var
      diff            = posterior_mean - prior_mean
      diff_term       = diff * diff / prior_var
      logvar_division = prior_logvar - posterior_logvar
      # put KLD together
      KLD = 0.5 * ( (var_division + diff_term + logvar_division).sum(1) - self.K)
#       print(KLD.mean())
      # loss
      loss = (NL + KLD)
      # in traiming mode, return averaged loss. In testing mode, return individual loss
      if avg:
          return loss.mean() # averaged over all the documents in the batch (1/batch_size)*sum
      else:
          return loss

def train():
    for epoch in range(args.epochs):
        all_indices = torch.randperm(doc_term_matrix_tensor.size(0)).split(args.batch_size)
        loss_epoch = 0.0
        model.train()                   # switch to training mode
        for batch_indices in all_indices:
            if not args.nogpu: batch_indices = batch_indices.cuda()
            docs = Variable(doc_term_matrix_tensor[batch_indices])
#             print(batch_indices.shape)
#             print(docs.shape)
            recon, loss = model(docs)
            # optimize
            optimizer.zero_grad()       # clear previous gradients
            loss.backward()             # backprop
            optimizer.step()            # update parameters
            # report
            loss_epoch += loss.data[0]    # add loss to loss_epoch, then take the average in the print statement
        if epoch % 5 == 0:
            print('Epoch {}, loss={}'.format(epoch, loss_epoch / len(all_indices)))       

Load training data (separate into batches)

In [0]:
train_data = torch.utils.data.TensorDataset(torch.tensor(doc_term_matrix))
train_loader = torch.utils.data.DataLoader(train_data,                                            
    batch_size = args.batch_size, shuffle = True)
#     batch_size = args.batch_size, shuffle = False)

instantiate model and define functions for training

In [0]:
model = VAE(vocab_size = doc_term_matrix.shape[1], 
            num_docs = doc_term_matrix.shape[0], 
            wordvec_dim = 100, 
            encoder_hidden = 256, 
            rp_normal_dim = 50, 
            num_samples = 1,
            K = 20)#.to(device) 
      
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas = (.99, .999))
# optimizer = optim.RMSprop(model.parameters(), lr = 1e-3)

# Reconstruction + KL divergence losses summed over all elements and batch
# def loss_function(data, train_loader, log_theta_dot_beta, x, mu, logvar, t):

# #     KLD = (-0.5) * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # KL divergence; it's fine to sum them all up now, rather than for each sample in mini-batch, because they'll all be summed up anyways
#     print(logvar.shape)
#     print(mu.shape)
#     print(data.shape)
#     print(log_theta_dot_beta.shape)
#     KLD = -.5 * (1 + logvar - mu.pow(2) - logvar.exp())
# #     BCE = log_theta_dot_beta.sum()
#     BCE = (data * log_theta_dot_beta).sum(1)

# #     print("BCE: " + "{:.2f}".format(float(BCE)))
# #     print("KLD: " + "{:.2f}".format(float(KLD)))
    
# #     return (-BCE + KLD) / len(train_loader.dataset)
#     return (-BCE + KLD).mean()

  
  
  



enc_variables = list(model.lin1.parameters()) + list(model.mean.parameters()) + list(model.logvar.parameters())
dec_variables = list(model.word_embedding.parameters()) + list(model.lin2.parameters()) + list(model.topic_embedding.parameters())

optim_enc = optim.Adam(enc_variables, lr=1e-3, betas = (.9, .999))
optim_dec = optim.Adam(dec_variables, lr=1e-3, betas = (.9, .999))

# def train(epoch):
#     model.train()
#     train_loss = 0
#     for switch in range(2):
#         if switch == 0:
#             print("updating encoder variables")
#             optimizer = optim_enc
#         else:
#             print("updating decoder variables")
#             optimizer = optim_dec
#         for batch_idx, data in enumerate(train_loader):
#     #         data = data.to(device)
#             optimizer.zero_grad()
#             log_theta_beta, mu, logvar, topic_vecs = model(data[0].float(), data[0].shape[0])
#             loss = loss_function(data[0].float(), train_loader, log_theta_beta, data, mu, logvar, topic_vecs)
#             loss.backward()
#             train_loss += loss.item()
#             optimizer.step()
#             if batch_idx % args.log_interval == 0:
#                 print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                     epoch, batch_idx * data[0].shape[0], len(train_loader.dataset),
#                     100. * batch_idx / len(train_loader),
#     #                 loss.item() / data[0].shape[0]))
#                     loss.item() ))

#         print('====> Epoch: {} Average loss: {:.4f}'.format(
#               epoch, train_loss / len(train_loader.dataset)))

#         return train_loss / len(train_loader.dataset)


# def test(epoch):
#     model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for i, (data, _) in enumerate(test_loader):
#             data = data.to(device)
#             recon_batch, mu, logvar = model(data)
#             test_loss += loss_function(recon_batch, data, mu, logvar).item()
#             if i == 0:
#                 n = min(data.size(0), 8)
#                 comparison = torch.cat([data[:n],
#                                       recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
# #                 save_image(comparison.cpu(),
# #                          'results/reconstruction_' + str(epoch) + '.png', nrow=n)

#     test_loss /= len(test_loader.dataset)
#     print('====> Test set loss: {:.4f}'.format(test_loss))

# if __name__ == "__main__":
#     for epoch in range(1, args.epochs + 1):
#         train(epoch)
#         test(epoch)
#         with torch.no_grad():
#             sample = torch.randn(64, 20).to(device)
#             sample = model.decode(sample).cpu()
#             save_image(sample.view(64, 1, 28, 28),
#                        'results/sample_' + str(epoch) + '.png')

 train the model

In [96]:
# losses = []
# for epoch in range(1, args.epochs + 1):
#     losses.append(train(epoch))
#     if epoch > 1:
#         if np.abs(losses[epoch-1] - losses[epoch-2]) < 1e-2:
#             break
train()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:117: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0, loss=23469.783203125
Epoch 5, loss=823431.8125
Epoch 10, loss=1775.6143798828125
Epoch 15, loss=588.0286254882812
Epoch 20, loss=573.9539794921875
Epoch 25, loss=2504.423095703125


In [0]:
associations = {
    'jesus': ['prophet', 'jesus', 'matthew', 'christ', 'worship', 'church'],
    'comp ': ['floppy', 'windows', 'microsoft', 'monitor', 'workstation', 'macintosh', 
              'printer', 'programmer', 'colormap', 'scsi', 'jpeg', 'compression'],
    'car  ': ['wheel', 'tire'],
    'polit': ['amendment', 'libert', 'regulation', 'president'],
    'crime': ['violent', 'homicide', 'rape'],
    'midea': ['lebanese', 'israel', 'lebanon', 'palest'],
    'sport': ['coach', 'hitter', 'pitch'],
    'gears': ['helmet', 'bike'],
    'nasa ': ['orbit', 'spacecraft'],
}
def identify_topic_in_line(line):
    topics = []
    for topic, keywords in associations.items():
        for word in keywords:
            if word in line:
                topics.append(topic)
                break
    return topics
def print_top_words(beta, feature_names, n_top_words=10):
    print('---------------Printing the Topics------------------')
    for i in range(len(beta)):
        line = " ".join([feature_names[j] 
                            for j in beta[i].argsort()[:-n_top_words - 1:-1]])
        topics = identify_topic_in_line(line)
        print('|'.join(topics))
        print('     {}'.format(line))
    print('---------------End of Topics------------------')

In [98]:
# emb = model.decoder.weight.data.cpu().numpy().T
# print("shape of beta is " + str(emb.shape))
print_top_words(model.beta.detach().numpy()
                .T, vectorizer.get_feature_names())

---------------Printing the Topics------------------
jesus|comp 
     point read research windows paul interested players fast open christianity

     use question help program non won technology window gun change

     host think team year sure available got let data sun

     like access true tell andrew value unix later version unless

     com thanks going org hear service points league uiuc control

     right car home wrong important numbers key couldn white sent

     state say did didn really david just bit list number
midea
     computer thing gov far post israel current common saying disk

     writes world way government great fact software distribution times send

     reply said university does distribution used just drive work run
jesus
     information right christian earth house matter care understand player john

     article posting know problem does just long public science course
jesus
     nntp university usa look game jesus man bad work just

     good new max day

In [90]:
model.parameters

<bound method Module.parameters of VAE(
  (word_embedding): Embedding(1739, 100)
  (topic_embedding): Embedding(20, 100)
  (lin1): Linear(in_features=1739, out_features=256, bias=True)
  (mean): Linear(in_features=256, out_features=50, bias=True)
  (logvar): Linear(in_features=256, out_features=50, bias=True)
  (lin2): Linear(in_features=50, out_features=20, bias=True)
  (dropout): Dropout(p=0.8)
)>

In [29]:
model.beta.shape

torch.Size([1739, 50])

In [19]:
# beta = F.softmax(model.word_embedding.weight.mm(model.topic_embedding.weight.t()))

beta = model.decoder.weight.softmax(1)
print(beta.sum(1).shape)
print(beta.shape)
_, ind = torch.sort(beta, 0)
# ind.numpy()[0:50, 0] - ind.numpy()[0:50, 1]
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:25, 0])
print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:25, 1])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 2])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 3])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 4])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 5])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 7])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 8])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 9])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 15])
# print(np.array(vectorizer.get_feature_names())[ind.numpy()][0:20, 19])

torch.Size([2441])
torch.Size([2441, 2])
['ins' 'ended' 'units' 'suggestion' 'model' 'bruce' 'jupiter' 'myers'
 'charles' 'cadlab' 'obviously' 'term' 'charge' 'ohio' 'morning' 'lies'
 'stated' 'began' 'machine' 'supposed' 'george' 'nation' 'signature'
 'loved' 'drugs']
['position' 'university' 'forms' 'success' 'trying' 'administrative'
 'fighting' 'specific' 'orbital' 'chances' 'opportunity' 'gnv' 'math'
 'like' 'car' 'weight' 'articles' 'worthwhile' 'exists' 'galaxy'
 'understand' 'known' 'estimate' 'cosmic' 'exactly']


##  Stuff

In [0]:
model.beta

In [46]:
# a = torch.randn(100, 128)
a = torch.tensor([[1,2,3], [4,5,6]]).float()
b = torch.tensor([[1,2,3], [4,5,6]]).float()

F.cosine_similarity(a, b)
b.t()


tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])

Get topic distributions

In [0]:
# unscaled_topics = torch.mm(model.word_embedding(torch.tensor(np.arange(doc_term_matrix.shape[1]))),
#          torch.transpose(model.topicslayer.weight, 0, 1))
# topic_dist = torch.softmax(unscaled_topics, dim = 0)
# topic_dist.sum(dim = 0)

This one helped us a lot

In [0]:
#model.encode(torch.LongTensor(doc_term_matrix[0]))
#input = torch.tensor(doc_term_matrix).float()
input = torch.tensor(doc_term_matrix).float()[[0, 1], ]
mu, sigma = model.encode(input)
z = model.reparameterize(mu, sigma)
# model.decode(x, input.shape[0])

x = model.fc3(z)
theta = F.softmax(x) # to get theta
embedding_matrix = model.word_embedding(torch.tensor(np.arange(14)))
word_dot_topic = model.fc4(embedding_matrix) # weights corresp to topic vector
beta = F.softmax(word_dot_topic)
log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(beta, 0, 1)))
#theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
# print(theta.shape)
# print(theta)
# print(embedding_matrix)
# print(word_dot_topic)
print(beta.shape)
print(beta)
print(log_theta_dot_beta)
print(torch.exp(log_theta_dot_beta_normalized))


In [0]:
#model.encode(torch.LongTensor(doc_term_matrix[0]))
#input = torch.tensor(doc_term_matrix).float()
input = torch.tensor(doc_term_matrix).float()[[0, 1], ]
print(input)
mu, sigma = model.encode(input)
z = model.reparameterize(mu, sigma)
print(z)
# model.decode(x, input.shape[0])


# x = model.lin2(z)
# theta = F.softmax(x) # to get theta
# embedding_matrix = model.word_embedding(torch.tensor(np.arange(model.num_docs)))
# word_dot_topic = model.topicslayer(embedding_matrix) # weights corresp to topic vector
# model.beta = F.softmax(word_dot_topic, dim = 0)
# log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(model.beta, 0, 1)))
# #theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
# log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
# print(embedding_matrix.shape) # dim of embedding matrix is 1544 x 100


x = model.lin2(z)
theta = F.softmax(x, 1) # to get theta
print(theta.sum(1))
embedding_matrix = model.word_embedding.weight
print(model.word_embedding(torch.tensor(np.arange(model.num_docs))).shape)
print(embedding_matrix.shape)
word_dot_topic = model.topicslayer(embedding_matrix) # weights corresp to topic vector
model.beta = F.softmax(word_dot_topic, dim = 0)
log_theta_dot_beta = torch.log(torch.mm(theta, torch.transpose(model.beta, 0, 1)))
#theta_dot_beta = torch.exp(log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0))
log_theta_dot_beta_normalized = log_theta_dot_beta - torch.logsumexp(log_theta_dot_beta, dim = 0)
# print(embedding_matrix.shape) # dim of embedding matrix is still 1544 x 100


In [0]:
###########################
###########################
###########################

test_input = torch.tensor(doc_term_matrix).float()[[0, 1], ] # pretend_batch_size = 2
# print(test_input.shape) # 2 x 2441, where 2441 is vocab size
mu, logvar = model.encode(test_input)
# print(mu.shape) # 2 x 50
# print(sigma.shape) # 2 x 50
z = model.reparameterize(mu, sigma) # 2 x 50
# print(z.shape) # 2 x 50
output = model.decode(z)
# print(output)
# print(output.shape)

pretend_num_docs = 50
pretend_batch_size = test_input.shape[0]
#print(output.sum())
BCE = pretend_num_docs * 1.0 / pretend_batch_size * output.sum() # we sum the log probabilities
# print(1 + logvar - mu.pow(2) - logvar.exp())
KLD0 = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
print(logvar)
#KLD1 = -0.5 * torch.sum(2 + torch.sum(torch.cumprod(logvar)) - torch.mm(torch.transpose(mu, 1, 0),mu) - logvar.exp()) # this is a number, should be 
############################################################################################################
#KLD2 = 0.5 * (torch.sum(logvar.exp()) + torch.dot(mu, mu) - 50 - torch.log(torch.cumprod(logvar))) ###########################
# print(float(BCE))

# print("BCE: " + "{:.2f}".format(float(BCE)))
# print("KLD: " + "{:.2f}".format(float(KLD)))
# print("Loss: " + "{:.2f}".format(float(- BCE + KLD)))
# return - BCE + KLD # - .1 * (zeta - nu)

In [0]:
# X'X
torch.mm(torch.transpose(torch.tensor([[1, 2], [3, 4]]), 1, 0),torch.tensor([[1, 2], [3, 4]]))

# Topic Coherence

In [0]:
from itertools import product

def topic_coherence(beta, M, doc_term_matrix):
  K = beta.shape[1] # beta has dim V x K
  coherences = np.zeros(K)
  for t in range(K):
    index = np.argsort(-beta[:, t])[0:M]
    cart_prod = product(list(index), list(index))
    for ind1, ind2 in cart_prod:
      if ind1 == ind2:
        pass
      else:
        d_ind1 = (doc_term_matrix[:, ind1] > 0).sum()
        d_ind12 = ((doc_term_matrix[:, ind1] > 0) & (doc_term_matrix[:, ind2] > 0)).sum()
        coherences[t] += np.log1p(d_ind12) - np.log(d_ind1)

  return coherences

In [16]:
topic_coherence(model.beta.detach().numpy(), 20, doc_term_matrix)

array([-5.05942546, -5.05942546, -5.05942546, ..., -5.05942546,
       -5.05942546, -5.05942546])